# TP - 1. Introduction à scikit-learn - classification

Dans ce notebook, vous allez apprendre :
* à vous servir d'un notebook Jupyter pour garder une trace de l'analyse de vos données ;
* à vous familiariser avec la librairie scikit-learn;
* à developper un premier algorithme de classification.

Ce noteboook utilise les librairies suivantes :
* python 3.7.7
* numpy 1.18.4
* matplotlib 3.2.1
* scikit-learn 0.23.1

Pour vérifier quelles versions de ces librairies vous utilisez, faites tourner la cellue ci-dessous en cliquant dessus puis en cliquant sur le bouton "Play" dans le menu au-dessus de cette fenêtre, ou en tapant Shift+Enter.

In [ ]:
import sys
print(sys.version)

import numpy
print(numpy.__version__)

import matplotlib
print(matplotlib.__version__)

import sklearn
print(sklearn.__version__)

# 1.  Le notebook Jupyter

Jupyter est une application web qui vous permet de créer et partager des documents appelés _notebooks_ (tels que ce notebook .ipynb) qui contient du code modifiable et exécutable, des visualisations, et du texte explicatoire qui peut être formaté avec une syntaxe markdown simple et contenir des équations.

Quelques éléments concernant l'utilisation des notebooks Jupyter :
* Chaque bloc éditable est contenu dans une cellule (_cell_). Un cellule peut contenit du texte brut (_raw text_), du code, ou du texte formatté avec la syntaxe markdown, comme cette cellule. Pour plus d'information sur la syntaxe markdown, suivez le [guide](http://jupyter-notebook.readthedocs.io/en/latest/examples/Notebook/Working%20With%20Markdown%20Cells.html) !
* Pour exécuter une cellule, il suffit de cliquer dessus et de taper Shift+Enter (ou d'utiliser le bouton Play dans la barre de menu).
* Pour créer une nouvelle cellule vide en-dessous de celle que vous allez exécuter, utilisez Alt+Enter au lieu de Shift+Enter.
* Le menu Insert vous permet aussi d'insérer de nouvelles cellules avant ou après la cellue courante.
* Si le notebook ne répond plus, vous pouvez le redémarrer par le menu Kernel --> Restart.

Quelques éléments concernant l'utilisation d'un notebook Jupyter avec Python :
* Une cellule de code Python se comporte comme un shell Python interactif (et en particulier comme ipython, sur lequel est basé Jupyter). En particulier : 
  * Tabulation permet d'auto-compléter le mot-clé que vous avez commencé à taper
  * Taper un point d'interrogation après le nom d'un objet charge l'aide interactive pour cette fonction.
* Jupyter a des commandes Python spéciales (des raccourcis, en quelque sorte) qui s'appellent des _magics_. Par exemple, `%bash` permet d'exécuter du code bash (donc comme si vous étiez dans un terminal), `%paste` permet de coller un block de code précédemment copié (depuis le notebook ou une autre application) en conservant son formatage (et en particulier les indentations), et `%matplotlib inline` permet d'importer la librairie de visualisation de matplotlib et d'afficher les graphiques créés non pas dans une nouvelle fenêtre mais à l'intérieur du notebook. Vous trouverez une liste complète de _magics_ sur http://ipython.readthedocs.io/en/stable/interactive/magics.html 


### Ressources 
* Pour en savoir plus sur le shell python interactif : http://ipython.readthedocs.io/en/stable/interactive/tutorial.html
* Pour en savoir plus sur Jupyter : https://jupyter.org/
* Python et Python Scientifique : http://www.scipy-lectures.org/
* Pour une introduction rapide aux différences entre shell python, shell python interactif, et notebook : https://www.youtube.com/watch?v=ULzWaZQa1Dc (en français)

In [ ]:
# On importe les packages classiques de calcul et de visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Gestion des données

Ce TP a pour but de vous familiariser avec scikit-learn, d'apprendre et d'utiliser les modèles proposés par cette librairie.

On utilisera une version simplifiée du classique jeu de donnée MNIST, digits, dont les élements sont des images de chiffres tracés à la main.
Ici chaque image est de dimension `8x8`.
Importons le.

In [ ]:
from sklearn import datasets
digits = datasets.load_digits()
print(digits.data.shape)
print(digits.target)

L'attribut `data` d'un dataset sklearn a systématiquement les dimensions `(n_samples, n_features)`.
Ici une ligne correspond donc à une image "aplatie". On peut les afficher en restructurant ce vecteur grace à la méthode `.reshape` d'un tableau numpy.

In [ ]:
# Plot des 16 premiers éléments du dataset.
for i in range(16):
    ax = plt.subplot(4, 4, i+1)
    ax.imshow(digits.data[i,:].reshape((8,8)), cmap='Greys')
    ax.set_axis_off()

`data` et `target` sont deux des attributs de ce jeu de données.
Il est possible de lister les attributs et méthodes d'une classe avec `dir(object)`.

**Question:**

> Y avait-il un moyen plus simple d'afficher les images ?

**Réponse**:


Le but sera ici d'apprendre à classifier ces image, les classes étant les numéros représentés (10 classes donc).

Pour évaluer le classifieur, il nous faut un jeu de donné de test indépendant de celui d'entrainement.
On peut séparer aléatoirement le jeu de données en un `train` et un `test` grâce à [`train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, label_train, label_test = train_test_split(digits.data, digits.target, 
                                                                    test_size=0.3, random_state=84)

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(label_test)
plt.xlabel('test labels')
plt.subplot(1, 2, 2)
plt.hist(label_train)
plt.xlabel('train labels')

Si la séparation est faite totalement aléatoirement et que la base de donnée n'est pas suffisamment grande, on peut avoir une répartition inégale des labels au sein du `test` et du `train`. 
Celà peut affecter les performances des modèles. On va en général chercher à séparer le jeu de données en conservant 
dans le `train` et le `test` la même proportion de labels que dans le jeu de données initial : c'est la stratification.

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test, label_train, label_test = train_test_split(digits.data, digits.target, 
                                                                    test_size=0.3, random_state=84, stratify=digits.target)

In [ ]:
plt.subplot(1, 2, 1)
plt.hist(label_test)
plt.xlabel('test labels')
plt.subplot(1, 2, 2)
plt.hist(label_train)
plt.xlabel('train labels')

## Apprentissage d'un classifieur

On peut maintenant entrainer un modèle sur le jeu d'entrainement (`data_train`, `label_train`) et le tester sur (`data_test`, `label_test`).

Les modèles de sklearn sont implémentés par une classe héritée de `BaseEstimator`.
On les utilise tous de la même manière, en 3 étapes:

* Instancier le modèle `model(**kwargs)`
* Apprendre le modèle sur le jeu d'entrainement avec la méthode `fit`: `model.fit(X=data, y=ground_truth)`.
* Utiliser le modèle, avec les méthodes `predict`, `score` par exemple: `model.predict(X=data_test)`

Rappel : on peut utiliser `dir(object)` pour lister les méthodes/attributs disponibles, et `help(object.methode)` pour afficher la doc de cette méthode.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

## Instanciez, apprennez et testez un classifieur des 5-plus-proches-voisins.



Il est possible de créer des pipelines d'execution grace à `sklearn.pipeline.Pipeline`.
De même qu'un autre modèle, une pipeline a des méthodes `fit`, `predict` etc...

Implémentons la régression logistique régularisée, avec comme paramètre de régularisation $\frac{1}{C} = \frac{1}{2}$.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

C = 2
penalty='l2'
lreg = LogisticRegression(penalty=penalty, C=C, max_iter=200)
pip = Pipeline([('scaler', StandardScaler()), ('lreg', lreg)])

# Apprenez et testez la regression logistique régularisée (précedeée d'une normalisation des données):



Regardons de plus près quelques exemples de succès et d'échec de l'algorithme 5-NNC

In [ ]:
np.random.seed(84)
## instanciez `preds` avec les prédictions du 5-NNC sur le set de test.
preds = 

success = np.array(preds) == np.array(label_test)   

# Quelques cas de prédictions justes:
data_s, pred_s, label_s =data_test[success,:], preds[success], label_test[success]
sample = np.random.choice(len(pred_s), size=16) # Echantillon aléatoire de 16 succès
plt.figure(figsize=(7, 7))
for i in range(16):
    ax = plt.subplot(4, 4, i+1)
    pred, ground_truth = pred_s[sample][i], label_s[sample][i]
    ax.imshow(data_s[sample,:][i,:].reshape((8,8)), cmap='Greys')
    ax.set_title('Pred: {}, Label: {}'.format(pred, ground_truth), color='b')
    ax.set_axis_off()

# Quelques cas de prédictions fausses:
data_f, pred_f, label_f =data_test[~success,:], preds[~success], label_test[~success]
sample = np.random.choice(len(pred_f),size=16)
plt.figure(figsize=(7, 7))
for i in range(16):
    ax = plt.subplot(4, 4, i+1)
    pred, ground_truth = pred_f[sample][i], label_f[sample][i]
    ax.imshow(data_f[sample,:][i,:].reshape((8,8)), cmap='Greys')
    ax.set_title('Pred: {}, Label: {}'.format(pred, ground_truth), color='r')
    ax.set_axis_off()

## Evaluation d'un algorithme de classification

A l'oeil, sur un certain nombre d'échantillons ça semble assez bien fonctionner.
On peut évaluer plus précisément les performances de cet algo. Pour ça, on va utiliser le module [`metrics`](https://scikit-learn.org/stable/modules/model_evaluation.html) de sklearn.
On y trouve déjà implémenté une série de métriques utiles: [`accuracy`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html#sklearn.metrics.accuracy_score), [`f1_score`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score), [`auc_roc`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_auc_score.html#sklearn.metrics.roc_auc_score).

Vous noterez que tous les classifieurs de sklearn ont la méthode `score` qui calcule le score `accuracy` moyen sur un dataset donné. 
    
De nombreuses autres métriques sont disponibles, pour évaluer la classification mais aussi d'autres tâches comme le clustering ou la régression.


## Métriques

In [ ]:
from sklearn import metrics
acc = metrics.accuracy_score(y_true=label_test, y_pred=preds)
f1_weighted = metrics.f1_score(label_test, preds, average='weighted')
print('Accuracy : {}, f1 : {}'.format(acc, f1_weighted))

Le rapport de classification fournit une vue synthétique des performances.

In [ ]:
report = metrics.classification_report(y_true=label_test, y_pred=preds)
print(report)

## Il est possible de générer un dictionnaire des performances :
## report_dict = metrics.classification_report(y_true=label_test, y_pred=preds, output_dict=True)
## On accède alors aux performances concernant le label 0 par report_dict['0'].

### Matrice de confusion

Les prédictions d'un algorithme de classification peuvent être visualisés à l'aide de la matrice de confusion.

In [ ]:
import seaborn as sns
cm = metrics.confusion_matrix(y_true=label_test, y_pred=preds)
ax = sns.heatmap(cm, cmap='coolwarm', annot=cm)
ax.set_xlabel('True label')
ax.set_ylabel('Predicted label')
plt.show()

In [ ]:
# Affichez la matrice de confusion de la regression logistique régularisée.




## Visualisation, sélection de modèle

### Visualisation des prédictions

Si l'on utilise les deux premières composantes principales du jeu de données pour apprendre un classifieur k-NN, 
on peut représenter comment se 'comporte' l'algorithme en prédisant les labels de points interpolés entre les points du jeu d'entrainement:

In [ ]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import RidgeClassifier
import matplotlib.patches as mpatches

# On calcul les composantes principales du jeu de données, on le projette ensuite sur ses deux premiers axes.
pca = PCA(n_components=2, whiten=True)
X = pca.fit_transform(data_train)

# On apprend un classifieur 1-NNC sur les deux premières composantes de chaque point de 'digits'
knnc = KNeighborsClassifier(n_neighbors=1)
knnc.fit(X=X, y=label_train)

## On veut représenter la prédiction du knn : 
## pour celà on prédit sur un ensemble assez dense de coordonnées autour des points d'entrainement.
## On colorera ces points en fonction de la prédiction du modèle

h = .02 # Définit l'écart entre les points de la grille
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
cmap = plt.cm.get_cmap('magma', 10)
Z = knnc.predict(np.c_[xx.ravel(), yy.ravel()]) #Prédiction d'un ensemble de points du plan des composantes principales.
Z = Z.reshape(xx.shape)
plt.figure(figsize=(10,10))
im=plt.pcolormesh(xx, yy, Z, cmap='magma',alpha=.8)
plt.legend([mpatches.Patch(color=cmap(l)) for l in range(10)], [str(l) for l in range(10)])

# On affiche un échantillon des points du jeu d'entrainement
sample = np.random.choice(X.shape[0], size=250)
plt.scatter(X[sample, 0], X[sample, 1], c=label_train[sample], cmap='magma', edgecolor='k', s=20)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())

On peut ainsi visualiser l'effet du paramètre `n_neighbors` sur la prédiction du modèle:

In [ ]:
plt.figure(figsize=(15,15))

# On va afficher les prédictions du plan des composantes principales en fonction de n_neighbors.
for o,k in enumerate([1, 5, 20, 50], 1):
    knnc = KNeighborsClassifier(n_neighbors=k)
    knnc.fit(X=X, y=label_train)
    Z = knnc.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    ax = plt.subplot(2, 2, o)
    ax.pcolormesh(xx, yy, Z, cmap='magma',alpha=.8, shading='auto')
    ax.set_title('k = {}'.format(k))
    ax.set_axis_off()

**Question:**

> Quelle est la classe d'hypothèse du 1-plus-proche-voisin ?

**Réponse:**

Essayez de représenter l'impact de $C$ sur l'apprentissage avec la régression logistique régularisée.

In [ ]:
plt.figure(figsize=(15,15))

# On va afficher les prédictions du plan des composantes principales en fonction de n_neighbors.


> Qu'observez-vous ? 

## Sélection de modèle

On cherche à trouver le meilleur hyper paramètre `C` ou `n_neighbors`.
Pour ce faire on peut effectuer une `grid search` : on va brutalement tester tous les hyperparamètres d'une liste (`grid` s'il y a plusieurs paramètres).
On cherche à choisir le paramètre qui maximise les performances du modèle en généralisation. Un proxy pour cette performance est la performance estiméee sur le `test set`.
Cependant, de la même façon qu'il est possible de surapprendre un jeu d'entrainement, il est possible de sur-apprendre un jeu de test par la sélection de l'hyperparamètre: l'hyperparamètre apporterait alors de bonnes performances pour ce jeu de test précis.

Pour éviter celà au maximum, on va utiliser une `validation croisée`.

![](crossval.001.jpeg)

Dans cette illustration, la meilleure performance moyenne est rapportée par les modèles paramétrés par $\text{n_neighbors}=2$. On sélectionne donc cet hyeraramètre la.

En pratique, on utilise [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html)

Comme les autres estimateurs de sklearn, il faut instancier GridSearchCV et utiliser sa méthode `fit`.

Les paramètres principaux du GridSearchCV sont :

* `estimator` : le modèle de classification utilisé
* `param_grid` : dictionnaire des hyperparamètres à tester. Les clés de ce dictionnaire sont de type `str` et correspondent exactement au nom du paramètre utilisé pour instancier `estimator`
* `scoring`: définit selon quelle mesure sont sélectionnés les hyperparamètres.
* `cv`: définit le nombre de sous-divisions du jeu de donnée

Comme une selection (et donc une forme d'apprentissage) est faite à l'aide du jeu de validation, il nous faut conserver un jeu de set finale, sur lequel on pourra calculer la vraie performance de généralisation du modèle.
Il convient donc d'appliquer la validation croisée sur `data_train`.

Une fois "fitté" on a accès aux résultats grace à l'attribut `.cv_results_`.
On peut aussi utiliser les méthodes `score` ou encore `predict`, qui reviennent à utiliser le meilleur modèle.

In [ ]:
from sklearn.model_selection import GridSearchCV

## Cherchez le meilleur paramètre n_neighbors pour l'algorithme
## des k-plus proches voisins: GridSearchCV pour le k-NNC

In [ ]:
## Représentez (d'une manière ou d'une autre) l'évolution des performances de validation 
## en fonction de n_neighbors.


In [ ]:
from sklearn.model_selection import GridSearchCV
stsc = StandardScaler()
data_train = stsc.fit_transform(data_train)
## Cherchez le meilleur paramètre C pour l'algorithme
## de la régression logistique régularisée

In [ ]:
## Représentez (d'une manière ou d'une autre) l'évolution des performances de validation 
## en fonction de C.


**Questions:** 
> Expliquez l'effet de ces deux hyperparamètres sur l'apprentissage.

**Réponse**:

On veut finalement choisir quel modèle utiliser entre le meilleur des k-NNC et des régressions logistiques régularisées. 

**Questions:**

> Comment comparer et sélectionner entre ces deux modèles ? \
> Comment finir d'évaluer le résultat de cette sélection ?

**Réponses:**

In [ ]:
# Sélection et évaluation du modèle final.